In [2]:
import pandas as pd
import numpy as np

In [3]:
def load_daily_max_prcp(filename):
    df = pd.read_csv(filename,parse_dates=['valid'],usecols=['valid','station','precip_in'])
    df = df.rename(columns={'valid':'datetime'})
    df['date'] = df['datetime'].dt.date
    df['date'] = df['date'].astype('datetime64[ns]')
    df['time'] = df['datetime'].dt.time
    df = df.groupby(['date','station'])['precip_in'].max()
    return df.unstack('station')

In [4]:
df_MA_hourly_max = load_daily_max_prcp('../asos/MA_hourly_prcp.csv')
df_CT_hourly_max = load_daily_max_prcp('../asos/CT_hourly_prcp.csv')
df_NY_hourly_max = load_daily_max_prcp('../asos/NY_hourly_prcp.csv')
df_NJ_hourly_max = load_daily_max_prcp('../asos/NJ_hourly_prcp.csv')
df_PA_hourly_max = load_daily_max_prcp('../asos/PA_hourly_prcp.csv')
df_DE_hourly_max = load_daily_max_prcp('../asos/DE_hourly_prcp.csv')

df_hourly_max = pd.concat([
    df_MA_hourly_max.loc['1970-01-01':'2022-12-31'][['BOS']],
    df_CT_hourly_max.loc['1970-01-01':'2022-12-31'][['BDL']],
    df_NY_hourly_max.loc['1970-01-01':'2022-12-31'][['JFK','LGA']],
    df_NJ_hourly_max.loc['1970-01-01':'2022-12-31'][['EWR']],
    df_PA_hourly_max.loc['1970-01-01':'2022-12-31'][['PHL']],
    df_DE_hourly_max.loc['1970-01-01':'2022-12-31'][['ILG']],
],axis=1,join='outer')

In [5]:
stations_all = pd.concat([
    pd.read_csv('../asos/MA_stations.csv'),
    pd.read_csv('../asos/CT_stations.csv'),
    pd.read_csv('../asos/NY_stations.csv'),
    pd.read_csv('../asos/NJ_stations.csv'),
    pd.read_csv('../asos/PA_stations.csv'),
    pd.read_csv('../asos/DE_stations.csv'),
])
stations = stations_all[['stid','station_name','lat','lon','elev']].set_index(['stid'])
stations = stations.loc[df_hourly_max.columns]
stations

,station_name,lat,lon,elev
station,,,,
BOS,BOSTON/LOGAN INTL,42.36057,-71.00973,9.0
BDL,HARTFORD/BRADLEY,41.93810,-72.68250,55.0
JFK,NEW YORK/JF KENNEDY,40.63861,-73.76222,7.0
LGA,New York/LaGuardia,40.77945,-73.88028,9.0
EWR,NEWARK INTL AIRPORT,40.68272,-74.16932,9.0
PHL,Philadelphia Intl,39.87335,-75.22663,2.0
ILG,WILMINGTON AIRPORT,39.67278,-75.60083,24.0


In [6]:
def locations(station_lat_long,units='km'):
    """
    https://en.wikipedia.org/wiki/Geographical_distance
    """

    if units == 'km':
        R = 6371.0
    else:
        R = 3958.76

    X = station_lat_long[['lat','lon']].copy()
    X = X * (np.pi/180.0)
    X['X'] = R * np.cos(X['lat'].mean()) * X['lon']
    X['Y'] = R * X['lat']
    return X[['X','Y']]

In [7]:
distances = pd.DataFrame(0, columns=stations.index, index=stations.index)

df_X = locations(stations)

for s1,(x1,y1) in df_X[['X','Y']].iterrows():
    for s2,(x2,y2) in df_X[['X','Y']].iterrows():
        distances.loc[s1,s2] = np.sqrt((x1-x2)**2+(y1-y2)**2)

distances

station,BOS,BDL,JFK,LGA,EWR,PHL,ILG
station,,,,,,,
BOS,0.000000,148.334073,300.435549,298.674016,324.707247,449.770335,488.308174
BDL,148.334073,0.000000,170.666265,163.550168,187.418026,313.852318,351.713757
JFK,300.435549,170.666265,0.000000,18.543623,34.591241,149.708196,188.281117
LGA,298.674016,163.550168,18.543623,0.000000,26.584593,151.576280,189.963286
EWR,324.707247,187.418026,34.591241,26.584593,0.000000,126.524603,164.648129
PHL,449.770335,313.852318,149.708196,151.576280,126.524603,0.000000,38.575098
ILG,488.308174,351.713757,188.281117,189.963286,164.648129,38.575098,0.000000
